**Instala pip install psycopg2 y pip install psycopg2-binary, para evitar el error de módulo no
instalado. Importa todas las bibliotecas necesarias.**

In [ ]:
import psycopg2
from psycopg2 import Error # Importar la clase Error
import os
import pandas as pd
from sqlalchemy import create_engine
from psycopg2 import sql

**Realiza la conexión con tu base de datos PostgreSQL.**

In [ ]:
def create_server_connection():
    conn = None
    try:
        conn = psycopg2.connect(database="datafundamental", 
                                user="datafundamental_owner", 
                                password="uozci0wQGpA6", 
                                host="ep-solitary-ponda2p3evpo.eu-cetral-1.aws.neon.tech", 
                                sslmode="require", 
                                port=5432)
        print("Database conn successful")
    except Error as err:
        print(f"Error: '{err}'")

    return conn
connection = create_server_connection()
connection.close()

**Crea estructura de carpetas y copia archivos CSV al directorio data.**

In [ ]:
# Crear estructura de carpetas
os.makedirs("repositorio/data", exist_ok=True)
os.makedirs("repositorio/src", exist_ok=True)
os.makedirs("repositorio/dist", exist_ok=True)

# Copiar archivos CSV al directorio `data`
data_files = {
"Caracteristicas_Equipos.csv": "/mnt/data/Caracteristicas_Equipos.csv",
"Historicos_Ordenes.csv": "/mnt/data/Historicos_Ordenes.csv",
"Registros_Condiciones.csv": "/mnt/data/Registros_Condiciones.csv",
}

for filename, path in data_files.items():
os.system(f"cp {path} repositorio/data/{filename}")

**Crea las tablas correspondientes.**

In [ ]:
def create_tables(engine):
    """Crea las tablas necesarias en PostgreSQL."""
    create_historicos_table = """
    CREATE TABLE IF NOT EXISTS historicos_ordenes (
        id_orden VARCHAR PRIMARY KEY,
        id_equipo VARCHAR NOT NULL,
        fecha DATE NOT NULL,
        tipo_mantenimiento VARCHAR NOT NULL,
        tiempo_reparacion_horas NUMERIC NOT NULL,
        costo_mantenimiento NUMERIC NOT NULL,
        ubicacion VARCHAR NOT NULL
);
"""
create_caracteristicas_table = """

CREATE TABLE IF NOT EXISTS caracteristicas_equipos (
    id_equipo VARCHAR PRIMARY KEY,
    tipo_equipo VARCHAR NOT NULL,
    fabricante VARCHAR NOT NULL,
    fecha_instalacion DATE NOT NULL,
    vida_util_anios NUMERIC NOT NULL,
    potencia_kw NUMERIC NOT NULL
);
"""
create_registros_table = """
CREATE TABLE IF NOT EXISTS registros_condiciones (
    id_equipo VARCHAR NOT NULL,
    fecha DATE NOT NULL,
    horas_operativas NUMERIC NOT NULL,
    temperatura_c NUMERIC NOT NULL,
    vibracion_mm_s NUMERIC NOT NULL,
    presion_bar NUMERIC NOT NULL,
    horas_acumuladas NUMERIC NOT NULL,
    PRIMARY KEY (id_equipo, fecha)
);
"""
with engine.connect() as connection:
    connection.execute(create_historicos_table)
    connection.execute(create_caracteristicas_table)
    connection.execute(create_registros_table)

**Cargar datos a las tablas PostgreSQL.**

In [ ]:
# Cargar datos a las tablas PostgreSQL
def load_data_to_postgresql(engine, csv_path, table_name):
    """Carga un archivo CSV en una tabla de PostgreSQL."""
    df = pd.read_csv(csv_path)
    df.to_sql(table_name, engine, if_exists="append", index=False)

# Ejecución
engine = create_database_connection()
create_tables(engine)

# Cargar datos
load_data_to_postgresql(engine, "repositorio/data/Historicos_Ordenes.csv", "historicos_ordenes")
load_data_to_postgresql(engine, "repositorio/data/Caracteristicas_Equipos.csv", "caracteristicas_equipos")
load_data_to_postgresql(engine, "repositorio/data/Registros_Condiciones.csv", "registros_condiciones")

**Verificar los resultados.**

In [ ]:
# Verificar resultados
historicos = pd.read_sql("SELECT * FROM historicos_ordenes LIMIT 5;", engine)
caracteristicas = pd.read_sql("SELECT * FROM caracteristicas_equipos LIMIT 5;", engine)
registros = pd.read_sql("SELECT * FROM registros_condiciones LIMIT 5;", engine)

import ace_tools as tools; tools.display_dataframe_to_user("Históricos de Órdenes", historicos)
tools.display_dataframe_to_user("Características de Equipos", caracteristicas)
tools.display_dataframe_to_user("Registros de Condiciones", registros)